In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.draw import disk
from scipy.fft import fft, ifft, fftfreq

def generate_sinogram(image_size, circle_center, circle_radius, num_angles):
    """
    生成简单几何体的CT模拟数据（sinogram）。

    Parameters:
        image_size (int): 图像的大小（方形边长）。
        circle_center (tuple): 圆的中心位置 (x, y)。
        circle_radius (int): 圆的半径。
        num_angles (int): 投影角度的数量。

    Returns:
        ndarray: Sinogram 数据。
        ndarray: 原始几何体图像。
    """
    # 创建空白图像
    image = np.zeros((image_size, image_size), dtype=np.float32)

    # 在图像中绘制圆形
    rr, cc = disk(circle_center, circle_radius, shape=image.shape)
    image[rr, cc] = 1.0

    # 定义投影角度（0到180度）
    theta = np.linspace(0., 180., num_angles, endpoint=False)

    # 生成 Radon 变换（Sinogram）
    sinogram = np.zeros((image_size, num_angles), dtype=np.float32)
    for i, angle in enumerate(theta):
        rotated_image = np.rot90(image, k=int(angle // 90))
        sinogram[:, i] = np.sum(rotated_image, axis=0)

    return sinogram, image

def ramp_filter(sinogram):
    """
    应用Ramp滤波器处理Sinogram数据。

    Parameters:
        sinogram (ndarray): Sinogram 数据。

    Returns:
        ndarray: 滤波后的Sinogram 数据。
    """
    num_projections, num_detectors = sinogram.shape

    # 创建频率范围
    freqs = fftfreq(num_detectors).reshape(-1, 1)
    ramp = np.abs(freqs)  # Ramp滤波器

    # 对Sinogram进行傅里叶变换、滤波并逆变换
    filtered_sinogram = np.zeros_like(sinogram)
    for i in range(num_projections):
        projection_fft = fft(sinogram[:, i])
        filtered_fft = projection_fft * ramp[:, 0]
        filtered_sinogram[:, i] = np.real(ifft(filtered_fft))

    return filtered_sinogram

def fbp_reconstruction(sinogram, theta):
    """
    使用FBP算法重建图像。

    Parameters:
        sinogram (ndarray): Sinogram 数据。
        theta (ndarray): 投影角度（与 Sinogram 匹配）。

    Returns:
        ndarray: 重建后的图像。
    """
    # 获取Sinogram尺寸
    num_detectors, num_projections = sinogram.shape
    image_size = num_detectors

    # 滤波Sinogram
    filtered_sinogram = ramp_filter(sinogram)

    # 初始化重建图像
    reconstructed_image = np.zeros((image_size, image_size), dtype=np.float32)

    # 逐角度进行反投影
    for i, angle in enumerate(theta):
        # 将滤波后的投影数据扩展为二维
        projection = filtered_sinogram[:, i]
        projection_2d = np.tile(projection, (image_size, 1))

        # 旋转到对应角度并累加
        rotated_projection = np.rot90(projection_2d, k=int(angle // 90))
        reconstructed_image += rotated_projection

    # 正规化
    reconstructed_image /= len(theta)

    return reconstructed_image

# 参数设置
image_size = 128
circle_center = (64, 64)
circle_radius = 30
num_angles = 180

# 生成 Sinogram 数据
sinogram, original_image = generate_sinogram(image_size, circle_center, circle_radius, num_angles)

# 投影角度
theta = np.linspace(0., 180., num_angles, endpoint=False)

# 使用 FBP 重建图像
reconstructed_image = fbp_reconstruction(sinogram, theta)

# 显示结果
plt.figure(figsize=(12, 6))

plt.subplot(1, 3, 1)
plt.title("Original Image")
plt.imshow(original_image, cmap='gray')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title("Sinogram")
plt.imshow(sinogram, cmap='gray', aspect='auto')
plt.xlabel("Projection Angle")
plt.ylabel("Detector Position")

plt.subplot(1, 3, 3)
plt.title("Reconstructed Image")
plt.imshow(reconstructed_image, cmap='gray')
plt.axis('off')

plt.tight_layout()
plt.show()
